In [53]:
import numpy as np
import pandas as pd
import os

# Load each .npy file and convert the list of dictionaries to a DataFrame
dataframes = []
for file in os.listdir('dataset'):
    if file.endswith('.npy'):
        data = np.load('dataset/' + file, allow_pickle=True)
        dataframes.append(pd.DataFrame.from_records(data))

# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(dataframes, ignore_index=True)
merged_df

,Word,Frame,X00,Y00,Z00,X01,Y01,Z01,X02,Y02,...,Z62,X63,Y63,Z63,X64,Y64,Z64,X65,Y65,Z65
0,Boy,1,-0.290488,-75.818914,-3.229877,-0.166410,-77.328566,-3.061451,-0.098528,-82.594196,...,-0.349284,0.610715,0.973082,-0.303378,0.405078,0.922159,-0.281479,0.559808,0.851646,-0.028510
1,Boy,2,-2.630732,163.577346,-2.202535,-2.630732,163.577346,-2.202535,-2.630732,163.577346,...,-0.346683,0.613981,0.946346,-0.395646,0.413621,0.845886,-0.267950,0.558445,0.851427,-0.036818
2,Boy,3,-2.649066,62.903229,-2.865200,-2.649066,62.903229,-2.865200,-2.649066,62.903229,...,-0.820756,0.612683,0.950004,-0.379825,0.448817,0.581546,-0.748774,0.557367,0.867107,-0.040340
3,Boy,4,-2.641445,73.387394,-6.816939,-2.641445,73.387394,-6.816939,-2.641445,73.387394,...,-0.892051,0.607402,0.961182,-0.300694,0.439404,0.449596,-0.823084,0.556838,0.878659,-0.022312
4,Boy,5,0.075973,-2.649841,-1.005892,0.190630,-5.362860,-1.022167,0.243538,-8.214614,...,-0.829807,0.604378,0.950633,-0.247957,0.445261,0.344964,-0.765876,0.552638,0.877294,-0.045415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14135,You,16,-0.255172,-14.686667,2.176972,-0.137746,-15.407824,3.206680,-0.088495,-16.454638,...,-0.189617,0.678464,0.430918,-0.998512,0.307135,0.832112,-0.110380,0.559508,0.746648,0.006331
14136,You,17,-0.247349,-18.125843,0.387121,-0.123823,-19.070908,-0.996675,-0.073921,-20.329048,...,-0.235708,0.673772,0.673639,-0.173289,0.301638,0.830727,-0.135007,0.558916,0.736754,0.016739
14137,You,18,-0.252560,-28.690787,5.913587,-0.133411,-30.150542,5.626362,-0.083474,-32.168021,...,-0.643596,0.653561,0.818583,-0.402061,0.293410,0.830436,-0.519518,0.550947,0.735413,0.027168
14138,You,19,-0.263622,-44.649560,6.507309,-0.145083,-46.797549,6.096631,-0.096428,-49.825290,...,-0.573453,0.627514,0.836671,-0.288298,0.281208,0.831862,-0.455801,0.543025,0.740520,0.026633


In [54]:
max_sequence_length = merged_df['Frame'].max()
max_sequence_length

20

In [55]:
def pad_sequences(group):
    # Calculate the number of padding rows needed
    padding_rows = max_sequence_length - len(group)
    
    # Create a DataFrame with padding rows filled with NaN (or any other padding value)
    padding_df = pd.DataFrame({
        'Word': [group['Word'].iloc[0]] * padding_rows,
        'Frame': np.arange(len(group) + 1, max_sequence_length + 1),
    })
    
    # Concatenate the original group with the padding DataFrame
    return pd.concat([group, padding_df], ignore_index=True)

# Group the DataFrame by 'Word' and apply the padding function
padded_df = merged_df.groupby('Word').apply(pad_sequences).reset_index(drop=True)
padded_df

,Word,Frame,X00,Y00,Z00,X01,Y01,Z01,X02,Y02,...,Z62,X63,Y63,Z63,X64,Y64,Z64,X65,Y65,Z65
0,Boy,1,-0.290488,-75.818914,-3.229877,-0.166410,-77.328566,-3.061451,-0.098528,-82.594196,...,-0.349284,0.610715,0.973082,-0.303378,0.405078,0.922159,-0.281479,0.559808,0.851646,-0.028510
1,Boy,2,-2.630732,163.577346,-2.202535,-2.630732,163.577346,-2.202535,-2.630732,163.577346,...,-0.346683,0.613981,0.946346,-0.395646,0.413621,0.845886,-0.267950,0.558445,0.851427,-0.036818
2,Boy,3,-2.649066,62.903229,-2.865200,-2.649066,62.903229,-2.865200,-2.649066,62.903229,...,-0.820756,0.612683,0.950004,-0.379825,0.448817,0.581546,-0.748774,0.557367,0.867107,-0.040340
3,Boy,4,-2.641445,73.387394,-6.816939,-2.641445,73.387394,-6.816939,-2.641445,73.387394,...,-0.892051,0.607402,0.961182,-0.300694,0.439404,0.449596,-0.823084,0.556838,0.878659,-0.022312
4,Boy,5,0.075973,-2.649841,-1.005892,0.190630,-5.362860,-1.022167,0.243538,-8.214614,...,-0.829807,0.604378,0.950633,-0.247957,0.445261,0.344964,-0.765876,0.552638,0.877294,-0.045415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14135,You,16,-0.255172,-14.686667,2.176972,-0.137746,-15.407824,3.206680,-0.088495,-16.454638,...,-0.189617,0.678464,0.430918,-0.998512,0.307135,0.832112,-0.110380,0.559508,0.746648,0.006331
14136,You,17,-0.247349,-18.125843,0.387121,-0.123823,-19.070908,-0.996675,-0.073921,-20.329048,...,-0.235708,0.673772,0.673639,-0.173289,0.301638,0.830727,-0.135007,0.558916,0.736754,0.016739
14137,You,18,-0.252560,-28.690787,5.913587,-0.133411,-30.150542,5.626362,-0.083474,-32.168021,...,-0.643596,0.653561,0.818583,-0.402061,0.293410,0.830436,-0.519518,0.550947,0.735413,0.027168
14138,You,19,-0.263622,-44.649560,6.507309,-0.145083,-46.797549,6.096631,-0.096428,-49.825290,...,-0.573453,0.627514,0.836671,-0.288298,0.281208,0.831862,-0.455801,0.543025,0.740520,0.026633


In [56]:
from sklearn.preprocessing import MinMaxScaler

data = padded_df.drop('Word', axis=1)
target = padded_df['Word']

# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))

features_scaled = scaler.fit_transform(data)

import pickle
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

###Random Forest

In [59]:
# Define the interval
interval = 20

# Initialize lists to hold the training and testing sets
X_train_list = []
y_train_list = []
X_test_list = []
y_test_list = []

# Iterate over features_scaled with a step of 20
for i in range(0, len(padded_df) - interval, 3*interval):
    # Slice the dataset for the current interval
    X_train = pd.DataFrame(features_scaled[i:i+2*interval])
    y_train = pd.DataFrame(target[i:i+2*interval])

    X_test = pd.DataFrame(features_scaled[i+2*interval:i+3*interval])
    y_test = pd.DataFrame(target[i+2*interval:i+3*interval])

    # Append the sliced DataFrames to the lists
    X_train_list.append(X_train)
    y_train_list.append(y_train)
    X_test_list.append(X_test)
    y_test_list.append(y_test)

# Convert the lists to DataFrames
X_train_df = pd.concat(X_train_list)
y_train_df = pd.concat(y_train_list)
X_test_df = pd.concat(X_test_list)
y_test_df = pd.concat(y_test_list)

X_train_df

,0,1,2,3,4,5,6,7,8,9,...,189,190,191,192,193,194,195,196,197,198
0,0.000000,0.952978,0.699372,0.454278,0.953955,0.713726,0.441782,0.954489,0.730702,0.435487,...,0.609107,0.485042,0.933159,0.577927,0.529268,0.860612,0.616883,0.373888,0.509505,0.514733
1,0.052632,0.934556,0.699717,0.454491,0.934556,0.714057,0.441959,0.934556,0.731019,0.435650,...,0.609978,0.488517,0.903483,0.534605,0.538125,0.776289,0.621676,0.368709,0.509020,0.489929
2,0.105263,0.934412,0.699572,0.454354,0.934412,0.713919,0.441823,0.934412,0.730890,0.435514,...,0.451326,0.487136,0.907543,0.542033,0.574620,0.484052,0.451347,0.364614,0.543859,0.479415
3,0.157895,0.934472,0.699587,0.453535,0.934472,0.713933,0.441012,0.934472,0.730903,0.434701,...,0.427466,0.481516,0.919951,0.579187,0.564860,0.338176,0.425023,0.362603,0.569525,0.533235
4,0.210526,0.955863,0.699477,0.454739,0.956765,0.713825,0.442201,0.957182,0.730798,0.435892,...,0.448296,0.478297,0.908241,0.603949,0.570932,0.222502,0.445288,0.346646,0.566492,0.464263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,0.789474,0.953256,0.699460,0.455399,0.954180,0.713811,0.443068,0.954568,0.730787,0.436809,...,0.662541,0.557140,0.331366,0.251541,0.427714,0.761062,0.677494,0.372749,0.276217,0.618748
36,0.842105,0.953318,0.699455,0.455028,0.954290,0.713806,0.442206,0.954683,0.730782,0.435825,...,0.647116,0.552147,0.600782,0.639008,0.422015,0.759530,0.668770,0.370499,0.254234,0.649819
37,0.894737,0.953277,0.699440,0.456174,0.954215,0.713791,0.443565,0.954608,0.730767,0.437247,...,0.510613,0.530639,0.761668,0.531593,0.413484,0.759208,0.532559,0.340221,0.251254,0.680956
38,0.947368,0.953190,0.699417,0.456297,0.954123,0.713768,0.443661,0.954506,0.730744,0.437335,...,0.534087,0.502919,0.781745,0.585008,0.400832,0.760785,0.555131,0.310119,0.262601,0.679359


In [60]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
clf.fit(X_train_df, y_train_df)

c:\Users\Mello\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(random_state=42)

In [61]:
from sklearn.metrics import classification_report

# Make predictions on the test set
y_pred = clf.predict(X_test_df)

# Evaluate the model
print(classification_report(y_test_df, y_pred))

              precision    recall  f1-score   support

         Boy       0.88      0.89      0.89       340
         Can       0.92      0.82      0.87       300
         Eat       0.87      0.96      0.91       260
        Fine       0.88      0.88      0.88       294
        Girl       0.86      0.89      0.87       226
        Help       0.99      0.99      0.99       300
         How       0.86      0.91      0.89       300
      Hungry       0.88      0.95      0.91       288
           I       0.98      0.96      0.97       232
      Mother       0.83      0.85      0.84       260
     Namaste       0.93      0.85      0.89       200
        Name       0.95      0.88      0.91       286
     Parents       0.98      0.98      0.98       294
      Sister       0.98      0.96      0.97       100
       Sleep       0.91      0.84      0.87       280
        This       0.87      0.88      0.88       320
         You       0.98      1.00      0.99       420

    accuracy              

In [16]:
import pickle

with open('rfmodel.pkl', 'wb') as file:
    pickle.dump(clf, file)


LSTM

In [36]:
# Define the number of time steps
time_steps = 20

# Prepare the data
X = []
y = []

for i in range(0, len(features_scaled) - time_steps, time_steps):
    X.append(features_scaled[i:i + time_steps])
    y.append(target[i])

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

# Reshape X to fit the LSTM input shape: [samples, time steps, features]
X = X.reshape((X.shape[0], X.shape[1], X.shape[2]))
len(X)

706

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [41]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'y_train' is your target variable with categorical values
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
num_classes = len(encoder.classes_)

model = Sequential()
model.add(LSTM(50, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(num_classes, activation='softmax')) # num_classes is the number of unique words you're predicting

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [48]:
history = model.fit(X_train, y_train_encoded, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
15/15 [==============================] - 0s 9ms/step - loss: 0.2160 - accuracy: 0.9313 - val_loss: 0.4930 - val_accuracy: 0.8230
Epoch 2/100
15/15 [==============================] - 0s 7ms/step - loss: 0.1887 - accuracy: 0.9490 - val_loss: 0.5273 - val_accuracy: 0.8496
Epoch 3/100
15/15 [==============================] - 0s 7ms/step - loss: 0.1599 - accuracy: 0.9601 - val_loss: 0.5128 - val_accuracy: 0.8407
Epoch 4/100
15/15 [==============================] - 0s 7ms/step - loss: 0.1779 - accuracy: 0.9557 - val_loss: 1.0857 - val_accuracy: 0.6637
Epoch 5/100
15/15 [==============================] - 0s 7ms/step - loss: 0.4920 - accuracy: 0.8426 - val_loss: 0.5843 - val_accuracy: 0.7965
Epoch 6/100
15/15 [==============================] - 0s 7ms/step - loss: 0.7582 - accuracy: 0.7672 - val_loss: 0.8771 - val_accuracy: 0.6283
Epoch 7/100
15/15 [==============================] - 0s 7ms/step - loss: 0.5273 - accuracy: 0.7805 - val_loss: 0.7532 - val_accuracy: 0.7345
Epoch 8/100
1

In [50]:
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


5/5 [==============================] - 0s 3ms/step - loss: 0.4925 - accuracy: 0.8310
Test Loss: 0.492537260055542, Test Accuracy: 0.8309859037399292


In [52]:
# Save the model as pickle file
with open('lstmmodel.pkl', 'wb') as file:
    pickle.dump(model, file)